In [1]:
#import packages
import pandas as pd
import numpy as np
from collections import Counter
from IPython.core.interactiveshell import InteractiveShell
import IPython.core.display as ipd
from sklearn.model_selection import KFold
import sys
import time

In [2]:
#set configurations
pd.set_option('max_columns', 1000)
pd.set_option('max_rows', 1000)
InteractiveShell.ast_node_interactivity = 'all'

In [3]:
#dataset
DSL = pd.read_csv('dados/DSL-StrongPasswordData.csv')
DSL.head(5)
DSL = DSL.drop(['sessionIndex', 'rep'], axis=1)

,subject,sessionIndex,rep,H.period,DD.period.t,UD.period.t,H.t,DD.t.i,UD.t.i,H.i,DD.i.e,UD.i.e,H.e,DD.e.five,UD.e.five,H.five,DD.five.Shift.r,UD.five.Shift.r,H.Shift.r,DD.Shift.r.o,UD.Shift.r.o,H.o,DD.o.a,UD.o.a,H.a,DD.a.n,UD.a.n,H.n,DD.n.l,UD.n.l,H.l,DD.l.Return,UD.l.Return,H.Return
0,s002,1,1,0.1491,0.3979,0.2488,0.1069,0.1674,0.0605,0.1169,0.2212,0.1043,0.1417,1.1885,1.0468,0.1146,1.6055,1.4909,0.1067,0.7590,0.6523,0.1016,0.2136,0.1120,0.1349,0.1484,0.0135,0.0932,0.3515,0.2583,0.1338,0.3509,0.2171,0.0742
1,s002,1,2,0.1111,0.3451,0.2340,0.0694,0.1283,0.0589,0.0908,0.1357,0.0449,0.0829,1.1970,1.1141,0.0689,0.7822,0.7133,0.1570,0.7877,0.6307,0.1066,0.1684,0.0618,0.1412,0.2558,0.1146,0.1146,0.2642,0.1496,0.0839,0.2756,0.1917,0.0747
2,s002,1,3,0.1328,0.2072,0.0744,0.0731,0.1291,0.0560,0.0821,0.1542,0.0721,0.0808,1.0408,0.9600,0.0892,0.6203,0.5311,0.1454,0.7195,0.5741,0.1365,0.2931,0.1566,0.1621,0.2332,0.0711,0.1172,0.2705,0.1533,0.1085,0.2847,0.1762,0.0945
3,s002,1,4,0.1291,0.2515,0.1224,0.1059,0.2495,0.1436,0.1040,0.2038,0.0998,0.0900,1.0556,0.9656,0.0913,1.2564,1.1651,0.1454,0.7550,0.6096,0.0956,0.1530,0.0574,0.1457,0.1629,0.0172,0.0866,0.2341,0.1475,0.0845,0.3232,0.2387,0.0813
4,s002,1,5,0.1249,0.2317,0.1068,0.0895,0.1676,0.0781,0.0903,0.1589,0.0686,0.0805,0.8629,0.7824,0.0742,0.8955,0.8213,0.1243,0.7632,0.6389,0.0430,0.1975,0.1545,0.1312,0.1582,0.0270,0.0884,0.2517,0.1633,0.0903,0.2517,0.1614,0.0818


In [28]:
user = 's002'

list_of_registered_users = np.setdiff1d(DSL['subject'].unique() , user)
list_of_registered_users = np.random.choice(list_of_registered_users, 4, replace=False)
treino = DSL[DSL['subject']==user][:20]

teste_impostor = list()
teste_genuino = DSL[DSL['subject']==user][20:40]
    
for impostor in list_of_registered_users:
    teste_impostor.append(DSL[DSL['subject']==impostor][:5])
teste_impostor = pd.concat(teste_impostor, ignore_index=True)
    
modelo = euclidian_treino(treino)
genuineScores = euclidian_score(modelo, teste_genuino.loc[:,teste_genuino.columns!='subject'])
impostorScores = euclidian_score(modelo, teste_impostor.loc[:,teste_genuino.columns!='subject'])
decision = calculate_best_threshold(userScores=genuineScores, impostorScores=impostorScores)

BAcc: 0.564516129032258  Decision: 0.0027154500000000003
BAcc: 0.6451612903225806  Decision: 0.17262630350000008
BAcc: 0.6612903225806451  Decision: 0.17316530949999995
BAcc: 0.5161290322580645  Decision: 0.0008105575000000002
BAcc: 0.6935483870967742  Decision: 0.01033044
BAcc: 0.6612903225806451  Decision: 0.0074361975
BAcc: 0.5483870967741935  Decision: 0.001757470000000001
BAcc: 0.6290322580645161  Decision: 0.0059640560000000006
BAcc: 0.6290322580645161  Decision: 0.005726398000000005
BAcc: 0.532258064516129  Decision: 0.0016317860000000027
BAcc: 0.7580645161290323  Decision: 0.5497970220000001
BAcc: 0.7419354838709677  Decision: 0.53564322
BAcc: 0.6290322580645161  Decision: 0.004897404000000004
BAcc: 0.7258064516129032  Decision: 2.572205108500001
BAcc: 0.7096774193548387  Decision: 2.4079370025
BAcc: 0.6451612903225806  Decision: 0.006207568499999987
BAcc: 0.7580645161290323  Decision: 0.7329362205000007
BAcc: 0.7580645161290323  Decision: 0.6823162679999999
BAcc: 0.62903225806

In [29]:
decision

0.5497970220000001

In [27]:
def euclidian_treino(dados):
    return dados.mean().to_frame('mean')

def euclidian_score(modelo, dados_teste):
    p = modelo.shape[0]
    n = dados_teste.shape[0]
    
    if (dados_teste.shape[1]!=p):
        raise Exception("Numero de features diferente")
    
    scores =list()
    for feature in modelo.index:
        scores.append(sum(np.subtract(dados_teste[feature], modelo.loc[feature])**2))
    return scores

def calculate_best_threshold(userScores, impostorScores):
    predictions = userScores + impostorScores
    labels = np.concatenate((np.ones(len(userScores)), np.zeros(len(impostorScores))))
    
    best_BAcc = -float("inf")
    for score in predictions:
        _, _, BAcc = reporter(scores=predictions, true_labels=labels, decision=score)
        print("BAcc: {}  Decision: {}".format(BAcc, score))
        if BAcc > best_BAcc:
            best_BAcc = BAcc
            decision = score
    return decision
    
#1 genuino, 0 impostor    
def reporter(scores, true_labels, decision):
    y_genuine = list()
    y_impostor = list()
    for i, score in enumerate(scores):
        if (score <= decision): #classifiquei como positivo
            if (true_labels[i] == 1):
                y_genuine.append(1)
            else:
                y_impostor.append(1)
        else:
            if (true_labels[i] == 1):
                y_genuine.append(0)
            else:
                y_impostor.append(0)
                
    FNMR =  1 - sum(y_genuine) / len(y_genuine)
    FMR = sum(y_impostor) / len(y_impostor)
    BAcc = 1- (FNMR + FMR)/2
    return FNMR, FMR, BAcc
    

In [19]:
import scipy.stats as ss

In [32]:
ss.describe(genuineScores)
ss.describe(impostorScores)

DescribeResult(nobs=31, minmax=(0.0008105575000000002, 2.572205108500001), mean=0.3054881144193549, variance=0.3839014712684204, skewness=2.9439136763094327, kurtosis=7.779692332065721)

DescribeResult(nobs=31, minmax=(0.005104586000000002, 11.517068257999998), mean=3.325080099999999, variance=14.373144603052946, skewness=0.8588140114657208, kurtosis=-0.6185651557444212)

__Implementacao Pessoal (Tirei da cabeça rs, nao lembro de ter visto em algum artigo)__

A ideia dessa implementação é aproveitar que os scores gerados pelas amostras genuinas estao num intervalo bem menor dos que o das amostras impostoras.

Amostras genuinas: minmax=(0.0008105575000000002, 2.572205108500001)  
Amostras impostoras: minmax=(0.005104586000000002, 11.517068257999998)

In [ ]:
class EuclidianClassifier(ClassificationAlgorithm):
    def train_user_model(self, user_features=None):
        try:
            user_features=user_features.drop('subject', axis=1)
        except:
            pass
        
        centroids = list()
        distance_train = list()
        
        if (self.normalize==True):
            self._normalize_params = dict()
            for col in user_features:
                self._normalize_params[col] = (np.mean(user_features[col]) , np.std(user_features[col]))
                user_features[col] = (user_features[col] - np.mean(user_features[col])) / np.std(user_features[col])

        len_features = user_features.shape[0]
        for feature in user_features:
            centroids.append(np.sum(user_features[feature] / len_features))
        for _, row in user_features.iterrows():
            distance_train.append(np.sqrt(sum((row-centroids)**2)))
        average_distance = np.mean(distance_train)
        std_distance = np.std(distance_train)
        user_model = EuclidianUserModel(features=user_features)
        user_model.init(model=(average_distance-std_distance, average_distance+std_distance), centroids=centroids)
        return user_model

    def test(self, genuine_user=None, test_stream=None, user_model=None, decision_threshold=None):
        list_of_scores = list()
        y_genuino = list()
        y_impostor = list()
        y_true = test_stream.loc[:, 'subject']
        test_stream=test_stream.drop('subject', axis=1)

        if self.normalize==True:
            for col in test_stream.columns:
                test_stream[col] = (test_stream[col] - self._normalize_params[col][0]) / self._normalize_params[col][1]

        for i, row in test_stream.iterrows():
            distance = np.sqrt(sum((row - user_model.centroids)**2))
            if ((distance >= user_model.model[0]) & (distance <= user_model.model[1])):
                if (y_true[i] == genuine_user):
                    y_genuino.append(1)
                    if self.adaptive != False: # ATUALIZAR O MODELO
                        AS = AdaptiveStrategy(trainFunction= EuclidianClassifier(name=self.name, adaptive=self.adaptive, normalize=self.normalize),
                        userModel= user_model, newData=row)
                        user_model = AS.run(strategy=self.adaptive)

                else:
                    y_impostor.append(1)
            else:
                if (y_true[i] == genuine_user):
                    y_genuino.append(0)
                else:
                    y_impostor.append(0)
            list_of_scores.append(distance)
        return y_genuino, y_impostor, user_model